# Notebook de experimentación - Gabriel Tumbaco

In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from housing_price_prediction.utils.paths import data_raw_dir
from housing_price_prediction.utils.paths import data_interim_dir

train_path = data_raw_dir() / "train.csv"
df_raw = pd.read_csv(train_path)

## Exploracion y limpieza de las siguientes variables

Variables que describen el terreno, la ubicación y el tipo general de la propiedad.

- MSSubClass
- MSZoning
- LotFrontage
- LotArea
- Street
- Alley
- LotShape
- LandContour
- Utilities
- LotConfig
- LandSlope
- Neighborhood
- Condition1
- Condition2
- BldgType
- HouseStyle

**Experimentacion con el dataset**

In [55]:
vars = [
    'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 
    'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 
    'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 
    'BldgType', 'HouseStyle', 'SalePrice' 
]

df = df_raw[vars].copy()

In [56]:
print(df.shape)
df.info()

(1460, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MSSubClass    1460 non-null   int64  
 1   MSZoning      1460 non-null   object 
 2   LotFrontage   1201 non-null   float64
 3   LotArea       1460 non-null   int64  
 4   Street        1460 non-null   object 
 5   Alley         91 non-null     object 
 6   LotShape      1460 non-null   object 
 7   LandContour   1460 non-null   object 
 8   Utilities     1460 non-null   object 
 9   LotConfig     1460 non-null   object 
 10  LandSlope     1460 non-null   object 
 11  Neighborhood  1460 non-null   object 
 12  Condition1    1460 non-null   object 
 13  Condition2    1460 non-null   object 
 14  BldgType      1460 non-null   object 
 15  HouseStyle    1460 non-null   object 
 16  SalePrice     1460 non-null   int64  
dtypes: float64(1), int64(3), object(13)
memory usage: 194.0+ KB


# Limpieza de las variables

In [57]:
df.isna()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,SalePrice
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1456,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1457,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1458,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


Dato curioso: En Pandas, *isnull* e *isna* son alias para la misma funcion, no hay diferencias 

In [58]:
print("Contador de nulos / na por variables")
df.isnull().sum()

Contador de nulos / na por variables


MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
Alley           1369
LotShape           0
LandContour        0
Utilities          0
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
SalePrice          0
dtype: int64

## Variables numéricas

#### Inspección Inicial
> Verificar variables numéricas que tienen una desviación estándar cercana a 0, puesto que esto indica que la variable no toma un rango amplio de valores y puede que no aporte significativamente al análisis

In [59]:
df.describe()

,MSSubClass,LotFrontage,LotArea,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,180921.195890
std,42.300571,24.284752,9981.264932,79442.502883
min,20.000000,21.000000,1300.000000,34900.000000
25%,20.000000,59.000000,7553.500000,129975.000000
50%,50.000000,69.000000,9478.500000,163000.000000
75%,70.000000,80.000000,11601.500000,214000.000000
max,190.000000,313.000000,215245.000000,755000.000000


Las variables numéricas en el dataset presentan una varianza mayor a cero, por lo que se consideran relevantes para la resolución del problema.

### Variable LotFrontage
LotFrontage: Linear feet of street connected to property

In [60]:
print(df['LotFrontage'].isnull().sum())

259


In [61]:
df['LotFrontage'].describe()

count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

Se podria imputar los valores faltantes con la mediana global, pero se investigó que podría ser buena idea agrupar por vecindario y observar como cambia la mediana.
Se elige la mediana porque es una medida de tendencia central que no es sensible a datos atípicos

In [62]:
print(df.groupby('Neighborhood')['LotFrontage'].median())

Neighborhood
Blmngtn    43.0
Blueste    24.0
BrDale     21.0
BrkSide    52.0
ClearCr    80.0
CollgCr    70.0
Crawfor    74.0
Edwards    65.5
Gilbert    65.0
IDOTRR     60.0
MeadowV    21.0
Mitchel    73.0
NAmes      73.0
NPkVill    24.0
NWAmes     80.0
NoRidge    91.0
NridgHt    88.5
OldTown    60.0
SWISU      60.0
Sawyer     71.0
SawyerW    66.5
Somerst    73.5
StoneBr    61.5
Timber     85.0
Veenker    68.0
Name: LotFrontage, dtype: float64


Se observa que la mediana cambia mucho dependiendo del vecindario, por lo tanto no seria una buena idea imputar los valores faltantes de la variables con la mediana global. Una mejor estrategia seria imputar los valores faltantes con los de su correspondiente vecindario.

In [63]:
neighb_median = df.groupby('Neighborhood')['LotFrontage'].transform('median')
print(neighb_median)

0       70.0
1       68.0
2       70.0
3       74.0
4       91.0
        ... 
1455    65.0
1456    80.0
1457    74.0
1458    73.0
1459    65.5
Name: LotFrontage, Length: 1460, dtype: float64


In [64]:
df['LotFrontage'] = df['LotFrontage'].fillna(neighb_median)
print(df['LotFrontage'].isnull().sum())

0


### Variable LotArea
LotArea: Lot size in square feet

In [65]:
df['LotArea'].describe()

count      1460.000000
mean      10516.828082
std        9981.264932
min        1300.000000
25%        7553.500000
50%        9478.500000
75%       11601.500000
max      215245.000000
Name: LotArea, dtype: float64

In [66]:
print(df['LotArea'].isnull().sum())

0


### Variable MSSubClass
MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

In [67]:
print(df['MSSubClass'].isna().sum())

0


Esta variable es numérica, pero internamente representa una categoría. Por ello, es importante transformar este número a otro tipo de dato, con el objetivo de no confundir luego al modelo.

In [68]:
df['MSSubClass'] = df['MSSubClass'].astype(str)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MSSubClass    1460 non-null   object 
 1   MSZoning      1460 non-null   object 
 2   LotFrontage   1460 non-null   float64
 3   LotArea       1460 non-null   int64  
 4   Street        1460 non-null   object 
 5   Alley         91 non-null     object 
 6   LotShape      1460 non-null   object 
 7   LandContour   1460 non-null   object 
 8   Utilities     1460 non-null   object 
 9   LotConfig     1460 non-null   object 
 10  LandSlope     1460 non-null   object 
 11  Neighborhood  1460 non-null   object 
 12  Condition1    1460 non-null   object 
 13  Condition2    1460 non-null   object 
 14  BldgType      1460 non-null   object 
 15  HouseStyle    1460 non-null   object 
 16  SalePrice     1460 non-null   int64  
dtypes: float64(1), int64(2), object(14)
memory usage: 194.0+ KB


## Variables categóricas
#### Inspección Inicial
> Verificar variables categóricas que tienen un solo subnivel

In [70]:
cat_vars = ['Alley', 
            'MSZoning', 
            'Street', 
            'LotShape', 
            'LandContour', 
            'Utilities', 
            'LotConfig', 
            'LandSlope', 
            'Neighborhood', 
            'Condition1', 
            'Condition2', 
            'BldgType', 
            'HouseStyle'
        ]

for col in cat_vars:
    print(f"Columna: {col}: {df[col].nunique()} subniveles")

Columna: Alley: 2 subniveles
Columna: MSZoning: 5 subniveles
Columna: Street: 2 subniveles
Columna: LotShape: 4 subniveles
Columna: LandContour: 4 subniveles
Columna: Utilities: 2 subniveles
Columna: LotConfig: 5 subniveles
Columna: LandSlope: 3 subniveles
Columna: Neighborhood: 25 subniveles
Columna: Condition1: 9 subniveles
Columna: Condition2: 8 subniveles
Columna: BldgType: 5 subniveles
Columna: HouseStyle: 8 subniveles


Es posible observar que la variable Street y la variable Utilities tienen pocos subniveles. Hay que tomar en cuenta esto para verificar si en realidad podrian aportar información al análisis o podríamos eliminar por tener una varianza casi nula.

### Variable Alley
Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access

In [71]:
print(df['Alley'].value_counts(dropna=False))

Alley
NaN     1369
Grvl      50
Pave      41
Name: count, dtype: int64


Entonces, la mayoría de casas no tienen callejón. Para proceder correctamente se imputará el valor NaN con el string None.

In [72]:
df['Alley'] = df['Alley'].fillna("None")
print(df['Alley'].value_counts(dropna=False))

Alley
None    1369
Grvl      50
Pave      41
Name: count, dtype: int64


### Variable MSZoning
MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density

In [73]:
print(df['MSZoning'].value_counts(dropna=False))

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64


### Variable Street
Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved

In [74]:
print(df['Street'].value_counts(dropna=False))

Street
Pave    1454
Grvl       6
Name: count, dtype: int64


El 99% de las observaciones recaen sobre una única categoría en la variable Street. Se procede a eliminar la variable del dataset.

In [75]:
df.drop('Street', axis=1, inplace=True)

### Variable Utilities
Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	

In [76]:
print(df['Utilities'].value_counts(dropna=False))

Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64


El 99% de las observaciones recaen sobre una única categoría en la variable Street. Se procede a eliminar la variable del dataset.

In [77]:
df.drop('Utilities', axis=1, inplace=True)

## Filas repetidas

In [78]:
print(f"Tamaño del data set antes de eliminar filas repetidas: {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Tamaño del data set luego de eliminar filas repetidas: {df.shape}")

Tamaño del data set antes de eliminar filas repetidas: (1460, 15)
Tamaño del data set luego de eliminar filas repetidas: (1458, 15)


## Data Interim

In [79]:
train_interim_path = data_interim_dir() / "train_cleaned.csv"
df.to_csv(train_interim_path, index=False)